# It looks like that the polarization vectors computed by phonopy includes the phase factor exp(iQd)

In [1]:
# some goodies
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np, histogram.hdf as hh, histogram as H
import os

## Phonon data

In [2]:
ls ../data/graphite/

FORCE_CONSTANTS  POSCAR  SPOSCAR


In [3]:
disp_dir = os.path.abspath('../data/graphite/')

In [4]:
# cat {disp_dir}/POSCAR
# cat {disp_dir}/SPOSCAR

## basis and reciprocal basis

In [5]:
basis = np.array(
    map(float,
        "2.44618916393	0.0	0.0	-1.22309458197	2.11846195843	0.0	0.0	0.0	6.65422904762".split())
    )
basis.shape = 3, 3
basis

array([[ 2.44618916,  0.        ,  0.        ],
       [-1.22309458,  2.11846196,  0.        ],
       [ 0.        ,  0.        ,  6.65422905]])

In [6]:
# atom positions
fractional_positions = np.array([
    [0, 0, .25],
    [0, 0, .75],
    [1./3, 2./3, 0.25],
    [2./3, 1./3, 0.75]
])

positions = np.dot(fractional_positions, basis)
positions.shape

(4, 3)

In [7]:
a1,a2,a3 = basis
v = np.dot(a1, np.cross(a2, a3))
b1 = 2*np.pi * np.cross(a2, a3)/v
b2 = 2*np.pi * np.cross(a3, a1)/v
b3 = 2*np.pi * np.cross(a1, a2)/v
Q_basis = np.array([b1, b2, b3])

In [8]:
inv_Q_basis = np.linalg.inv(Q_basis)

## Compute phonons

In [9]:
from phonopy.interface import vasp
from phonopy.units import VaspToTHz
from phonopy import Phonopy, file_IO

bulk = vasp.read_vasp(os.path.join(disp_dir, 'POSCAR'), ['C'])
phonon = Phonopy(bulk, supercell_matrix=np.diag([6,6,1]), factor=VaspToTHz)
phonon.generate_displacements(distance=0.01)
force_constants = file_IO.parse_FORCE_CONSTANTS(os.path.join(disp_dir, 'FORCE_CONSTANTS'))
phonon.set_force_constants(force_constants)

## Polarization vectors: phase

It looks like the pol vectors from phonopy includes the phase factor. let us check

In [10]:
hkls2 = [[0,0,0], [0,0,1]]
phonon.set_qpoints_phonon(hkls2, is_eigenvectors=True, write_dynamical_matrices=False)
freqs2, pols2 = phonon.get_qpoints_phonon()
print pols2.shape
pols2 = np.transpose(pols2, (0,2,1))
pols2.shape = 2, 12, 4, 3

(2, 12, 12)


In [11]:
print freqs2
assert (freqs2[0] == freqs2[1]).all()

[[  3.34225831e-02   3.34225831e-02   9.28652901e-02   1.52100901e+00
    1.52100901e+00   2.61040097e+00   2.63887591e+01   2.65558347e+01
    4.76904068e+01   4.76904068e+01   4.78045328e+01   4.78045328e+01]
 [  3.34225831e-02   3.34225831e-02   9.28652901e-02   1.52100901e+00
    1.52100901e+00   2.61040097e+00   2.63887591e+01   2.65558347e+01
    4.76904068e+01   4.76904068e+01   4.78045328e+01   4.78045328e+01]]


Energies match apparantly. Next: polarizations

In [12]:
pols2[0][2]

array([[ -6.93685038e-12+0.j,   4.35802460e-12+0.j,  -5.00000874e-01+0.j],
       [ -6.89630367e-12+0.j,   4.37933976e-12+0.j,  -5.00000874e-01+0.j],
       [ -6.93700313e-12+0.j,   4.35832020e-12+0.j,  -4.99999126e-01+0.j],
       [ -6.89659952e-12+0.j,   4.37929682e-12+0.j,  -4.99999126e-01+0.j]])

In [13]:
pols2[1][2]

array([[ -6.93685038e-12 +0.00000000e+00j,
          4.35802460e-12 -3.43326754e-27j,
         -5.00000874e-01 -1.55037388e-16j],
       [  6.89630367e-12 -6.12376232e-27j,
         -4.37933976e-12 +7.34336765e-27j,
          5.00000874e-01 -2.89052597e-16j],
       [ -6.93700313e-12 -2.73356413e-32j,
          4.35832020e-12 -3.43332411e-27j,
         -4.99999126e-01 -1.55036847e-16j],
       [  6.89659952e-12 -6.12398803e-27j,
         -4.37929682e-12 +7.34338645e-27j,
          4.99999126e-01 -2.89051587e-16j]])

They don't match. for q=000, it is -.5, -.5, -.5, -.5. but for q=001, it is -.5, .5, -.5, .5

Next we adjust the phase factor and see if they match

In [14]:
np.allclose(
    pols2[1][2] * np.exp(-1j*np.pi*2* np.dot(hkls2[1], fractional_positions.T))[:, np.newaxis]  * 1j,
    pols2[0][2],
)

True

It matches! Note that an overall extra phase factor is fine (1j)